In [12]:
import os
import pandas as pd
import numpy as np

# Display Data about All Wildfire Incidents in Indonesia
Gather information for all wildfire events in Sumatra

In [13]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)


In [18]:
wf_data  = pd.read_csv("../data-total.csv")
wf_data.columns
# rename columns
wf_data.rename(columns={'Lat': 'lat', 
                        'Long': 'long', 
                        'Satellite': 'satellite', 
                        'Time_UTC': 'time_utc', 
                        'Date': 'date', 
                        'Source': 'source', 
                        'PROVINSI': 'provinsi',
                        'KAB_KOTA': 'kab_kota'}, inplace=True)
# adding '-' to date column values
wf_data.date = pd.to_datetime(wf_data.date, format="%Y%m%d")
wf_data.date = wf_data.date.astype('string')

In [19]:
# All provinces in sumatera
sumatera_provinces = [
    'SUMATERA UTARA',
    'JAMBI',
    'RIAU',
    'SUMATERA BARAT',
    'SUMATERA SELATAN',
    'BENGKULU',
    'LAMPUNG',
    'ACEH'
]
# create filter so that only events in sumatera will be returned.
sumatera_filter = wf_data['provinsi'].apply(lambda x: x in sumatera_provinces) 
sumatera_only = wf_data.loc[sumatera_filter]

In [20]:
display(sumatera_only.info())
display(sumatera_only.provinsi.unique())
display(sumatera_only.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7839 entries, 1 to 39670
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   lat        7839 non-null   float64
 1   long       7839 non-null   float64
 2   satellite  7839 non-null   object 
 3   time_utc   7839 non-null   object 
 4   date       7839 non-null   string 
 5   source     7839 non-null   object 
 6   provinsi   7839 non-null   object 
 7   kab_kota   7839 non-null   object 
 8   kec2006    7830 non-null   object 
 9   desaa2006  7830 non-null   object 
 10  nama_kaw   930 non-null    object 
dtypes: float64(2), object(8), string(1)
memory usage: 734.9+ KB


None

array(['LAMPUNG', 'RIAU', 'SUMATERA UTARA', 'SUMATERA BARAT',
       'SUMATERA SELATAN', 'ACEH', 'JAMBI', 'BENGKULU'], dtype=object)

,lat,long,satellite,time_utc,date,source,provinsi,kab_kota,kec2006,desaa2006,nama_kaw
1,-5.06259,105.101,LPN-NPP,06:11:04 AM,2016-08-24,LAPAN,LAMPUNG,LAMPUNG TENGAH,ANAK TUHA,BUMI JAYA,NaN
4,1.18991,100.672,LPN-NPP,06:29:59 AM,2016-08-23,LAPAN,RIAU,ROKAN HULU,KECAMATAN KAPENUHAN,UPT SP IV KOTO TENGAH,NaN
5,2.31418,100.288,LPN-NPP,06:29:59 AM,2016-08-23,LAPAN,SUMATERA UTARA,LABUHANBATU,PANAI TENGAH,SELAT BETING,NaN
6,1.41833,100.752,LPN-NPP,06:29:59 AM,2016-08-23,LAPAN,RIAU,ROKAN HILIR,KECAMATAN PUJUD,SIARANG ARANG,NaN
8,1.38958,100.673,LPN-NPP,06:36:36 AM,2016-09-13,LAPAN,RIAU,ROKAN HILIR,KECAMATAN PUJUD,TELUK NAYANG,NaN


In [21]:
sumatera_only.to_csv('sumatera.csv')

# Install Google Earth Engine API
Installation command can be done using pip command.
Pip Command: "pip install earthengine-api" (for upgrading: "pip install earthengine-api --upgrade")

In [22]:
!pip install earthengine-api
!pip install earthengine-api --upgrade
!pip install folium

     -------------------------------------- 232.8/232.8 KB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for earthengine-api: filename=earthengine_api-0.1.297-py3-none-any.whl size=263802 sha256=e50073a0e2b320bad41658d03e36ec64e97357ea921cfcb353c773347619711e
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\c1\03\3a\3e5a99e9ccbc825d5f7f4c6c74ec58c963aba2d1ac4addcc85
Successfully built earthengine-api
  Attempting uninstall: earthengine-api
    Found existing installation: earthengine-api 0.1.293
    Uninstalling earthengine-api-0.1.293:
      Successfully uninstalled earthengine-api-0.1.293


# Import earth engine API which is called ee


In [23]:
import ee
import folium
from folium import plugins

In [7]:
# Authenticate earth engine servers
ee.Authenticate()

Enter verification code: 4/1AX4XfWjBtx_dwxqoE5UGdF9UbMmyEbSZQ0nkUzagH3uikfh78azpZPs8Ue4

Successfully saved authorization token.


In [8]:
#initialize API
ee.Initialize()

In [24]:
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.ImageCollection(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

In [25]:
# def applyScaleFactors(collection_name):
#     opticalBands = collection_name.select('SR_B.').multiply(0.0000275).add(-0.2)
#     thermalBands = collection_name.select('ST_B6').multiply(0.00341802).add(149.0)
#     return collection_name.addBands(opticalBands, None, True).addBands(thermalBands, None, True)

# cloud masking
def cloudMask(collection_name):
    qa_band = collection_name.select('BQA')
    cloud_or_shadow = qa_band.bitwiseAnd(1<<4)\
                      .And(qa_band.bitwiseAnd(1<<6))\
                      .Or(qa_band.bitwiseAnd(1<<8))\
                      .Or(qa_band.bitwiseAnd(1<<12))
    masked = collection_name.mask()
    return collection_name.updateMask(cloud_or_shadow.Not())\
                          .updateMask(masked) 

# Calculate Normalized Burned Ratio
def calculateNBR(collection_name, band_nir_swir):
    nir  = collection_name.select(band_nir_swir[0])
    swir = collection_name.select(band_nir_swir[1])
    return nir.subtract(swir).divide(nir.add(swir))

# Satellite Imagery Discovery
Date must be long enough for the satellite to capture the image, e.g. 16 days, 24 days, or evenr 365.


In [27]:
####################################################lANDSAT#####################################################################
row = 836
coordinates = [float(sumatera_only.lat.values[row]), float(sumatera_only.long.values[row])]

# filter image collection
# set base date
base_date = ee.Date(sumatera_only.date.values[row])

# set geometry point
point = ee.Geometry.Point(sumatera_only.long.values[row], sumatera_only.lat.values[row])

# pre fire
prefire_date_start = ee.Date(base_date.advance(-365, 'day'))
prefire_date_end = ee.Date(base_date.advance(-1, 'day'))

# 'LANDSAT/LC08/C01/T1_TOA'

pre_landsat_collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA')\
                       .filterDate(prefire_date_start, prefire_date_end)\
                       .filterMetadata('CLOUD_COVER_LAND', 'less_than', 15)\
                       .filterBounds(point)\
                       .map(cloudMask)\
                       .mosaic()


#                        .map(cloudMask)\
#                        .mosaic()\
#                        .filterDate(prefire_date_start, prefire_date_end)\
#                        .filterBounds(point)\
#                        .filterMetadata('IMAGE_QUALITY', 'equals', 9)\
#                        .map(cloudMask)\
#                        .mosaic()\
#                        .sort("CLOUD_COVER")\
#                        .first()

# Calculate NBR for pre fire
pre_nbr = calculateNBR(pre_landsat_collection, ['B5', 'B7'])

# post fire
postfire_date_start = ee.Date(base_date.advance(0, 'day'))
postfire_date_end = ee.Date(base_date.advance(365, 'day'))

post_landsat_collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA')\
                       .filterDate(postfire_date_start, postfire_date_end)\
                       .filterBounds(point)\
                       .map(cloudMask)\
                       .mosaic()
#                        .map(cloudMask)\
#                        .mosaic()\
#                        .map(cloudMask)\
#                        .mosaic()\
#                        .sort("CLOUD_COVER")\
#                        .first()

# Calculate NBR for post fire
post_nbr = calculateNBR(post_landsat_collection, ['B5', 'B7'])

# calculate delta NBR
delta_nbr = pre_nbr.subtract(post_nbr).rename('DELTA_NBR')
post_landsat_collection = post_landsat_collection.addBands(delta_nbr)


map_landsat = folium.Map(location=coordinates, zoom_start=12.5)
folium.Marker(coordinates, popup=sumatera_only.desaa2006).add_to(map_landsat)

image_viz_params = {
    'bands': ['DELTA_NBR'],
    'min': 0.0,
    'max': 0.3,
}

map_landsat.add_ee_layer(post_landsat_collection, image_viz_params, None)

display(map_landsat)


# Data Preprocessing
This data preprocessing will involve:
1. 

In [1]:
####################################################SENTINEL#####################################################################
row = 111
coordinates = [float(sumatera_only.lat.values[row]), float(sumatera_only.long.values[row])]

nir = 'B8'
nir = 'B12'
# filter image collection
# set base date
base_date = ee.Date(sumatera_only.date.values[row])

# set geometry point
point = ee.Geometry.Point(sumatera_only.long.values[row], sumatera_only.lat.values[row])

# pre fire
prefire_date_start = ee.Date(base_date.advance(-365, 'day'))
prefire_date_end = ee.Date(base_date.advance(-1, 'day'))

# 'LANDSAT/LC08/C01/T1_TOA'

pre_landsat_collection = ee.ImageCollection('COPERNICUS/S2')\
                           .filterDate(prefire_date_start, prefire_date_end)\
                           .filter(ee.Filter.lessThanOrEqualsss('CLOUDY_PIXEL_PERCENTAGE', 10))\
                           .map(maskS2clouds)\
                           .mosaic()


# Calculate NBR for pre fire
pre_nbr = calculateNBR(pre_landsat_collection, [nir, swir])

# post fire
postfire_date_start = ee.Date(base_date.advance(1, 'day'))
postfire_date_end = ee.Date(base_date.advance(365, 'day'))

post_landsat_collection = ee.ImageCollection('COPERNICUS/S2')\
                           .filterDate(postfire_date_start, postfire_date_end)\
                           .filter(ee.Filter.lessThanOrEqualsss('CLOUDY_PIXEL_PERCENTAGE', 10))\
                           .map(maskS2clouds)\
                           .mosaic()


# # Calculate NBR for post fire
post_nbr = calculateNBR(post_landsat_collection, [nir, swir])

# # calculate delta NBR
delta_nbr = pre_nbr.subtract(post_nbr).rename('DELTA_NBR')
post_landsat_collection = post_landsat_collection.addBands(delta_nbr)


map_landsat = folium.Map(location=coordinates, zoom_start=12)
folium.Marker(coordinates, popup=sumatera_only.desaa2006).add_to(map_landsat)

visualization = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['DELTA_NBR', 'B4','B2'],
}

map_landsat.add_ee_layer(post_landsat_collection, visualization, None)
display(map_landsat)

NameError: name 'sumatera_only' is not defined